In [2]:
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

bokeh.io.output_notebook()


import img_seg_package.single_image as si

Loading BokehJS ...

ModuleNotFoundError: No module named 'cv2'

The purpose of this notebook is to provide a method for generating the associated figures of the CAP-A4 manuscript.

## Figure 2:

First, we want to generate an overview of the lung. To do this, we can load in an image from...

**Make an overview of the lung for Figure 2**

Next, we want to generate all the individual figures to demonstrate the workflow of the figure. To do this, we need to identify the location of the specific tissue image we want to examine. 

In [1]:
# The overall directory for the examined images
data_dir = '../Lung_Paper_Images/Test_Images/'

# The tissue examined for filename purposes
tissue = 'Lung'

# The variant being examined
variant = 'CAPA4'

# The animal being examined (integer for dataframe purposes)
animal = 2

# The replicate being examined (integer for dataframe purposes)
replicate = 2

# The name of the image being examined
img_loc = '1_XY04_00065_Overlay'

# The complete filename of the image being examined 
fname = data_dir + tissue + '/' + variant + '/' + str(animal) + '/' + str(replicate) + '/' + img_loc + '.tif'

In [1]:
inds = (df['Virus'] == variant) & (df['Animal'] == animal) & (df['Replicate'] == replicate)
    
im_af = skimage.img_as_float(skimage.io.imread(fname)[:,:,0])
im_sig = skimage.img_as_float(skimage.io.imread(fname)[:,:,1])

#Save the intermediate images for the figure
#skimage.io.imsave()

NameError: name 'df' is not defined

In [9]:


im_sub = im_sig - im_af*df.loc[inds, 'Image Multiplication Factor'].values[0]

im_height = np.shape(im_sub)[0]
im_width = np.shape(im_sub)[1]

template = np.load('../template.npy')

template_height = np.shape(template)[0]
template_width = np.shape(template)[1]

threshed_single = im_af > df.loc[inds, 'Area Threshold'].values[0]

im_bg = skimage.filters.gaussian(im_sub, df.loc[inds, 'Gaussian Size'].values[0], truncate = df.loc[inds, 'Truncation'].values[0])

im_no_bg = im_sub - im_bg

im_no_bg = (im_no_bg - df.loc[inds, 'Minimum Pixel Value'].values[0]) / (df.loc[inds, 'Maximum Pixel Value'].values[0] - df.loc[inds, 'Minimum Pixel Value'].values[0])

im_feat = skimage.feature.match_template(im_sub, template)

binary_thresh = im_feat > df.loc[inds, 'Size Threshold'].values[0]

binary_gauss = im_no_bg[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))] > df.loc[inds, 'Applied Threshold'].values[0]

binary_mult = binary_gauss*binary_thresh

binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=df.loc[inds, 'Minimum Size'].values[0])

im_labeled, n_labels = skimage.measure.label(binary_rem, background=0, return_num=True)

# Load phase image
im_p = skimage.img_as_float(skimage.io.imread(fname))[:,:,:2][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]

binary_rgb = np.dstack((binary_rem, binary_rem, binary_rem))